In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyIsoPEP.IsotonicPEP import IsotonicPEP

In [41]:
file_mapping = {
    "output/run1/splinePEP/0/peptide.target.txt": "PXD013274",
    "/home/yuqizh/quickt/PXD004467/percolator.target.peptides.txt": "PXD004467",
    "/home/yuqizh/quickt/PXD003868/percolator.target.peptides.txt": "PXD003868",
    "/home/yuqizh/quickt/PXD004565/percolator.target.peptides.txt": "PXD004565",
    "/home/yuqizh/quickt/PXD005025/percolator.target.peptides.txt": "PXD005025",
    "/home/yuqizh/quickt/PXD004536/percolator.target.peptides.txt": "PXD004536",
    "/home/yuqizh/quickt/PXD004948/percolator.target.peptides.txt": "PXD004948",
    "/home/yuqizh/quickt/PXD004325/percolator.target.peptides.txt": "PXD004325",
    "/home/yuqizh/quickt/PXD004424/percolator.target.peptides.txt": "PXD004424",
    "/home/yuqizh/quickt/PXD004947/percolator.target.peptides.txt": "PXD004947"
}

In [42]:
columns_to_drop = ["PSMId", "filename", "peptide", "proteinIds"]
methods = ["spline", "d2pep_pava", "d2pep_ispline", "q2pep_pava", "q2pep_ip_ispline", "q2pep_ip_pchip", "q2pep_ispline"]

label_mapping = {
    'spline': {'label': 'Spline', 'linestyle': '-'},
    'd2pep_pava': {'label': '$f_D$: PAVA', 'linestyle': '-'},
    'd2pep_ispline': {'label': '$f_D$: I-Spline', 'linestyle': '-'},
    'q2pep_pava': {'label': '$q$: PAVA', 'linestyle': '-'},
    'q2pep_ip_ispline': {'label': "$q$: ipPAVA", 'linestyle': '-'},
    'q2pep_ip_pchip': {'label': "$q$: ipPAVA (PCHIP)", 'linestyle': '-'},
    'q2pep_ispline': {'label': '$q$: I-Spline', 'linestyle': '-'}
}

def estimate_q(df, pep_col):
    df = df.copy()
    df['orig_order'] = range(len(df))
    df_sorted = df.sort_values(by="q-value", kind="mergesort")
    df_sorted[f"q_{pep_col}"] = df_sorted[pep_col].cumsum() / np.arange(1, len(df_sorted) + 1)
    df_sorted = df_sorted.sort_values(by='orig_order').drop(columns='orig_order')
    return df_sorted

def cal_max_rel_diff(q_value, q_est):
    relative_difference = np.abs(q_est - q_value) / np.where(q_value != 0, q_value, np.nan)
    return np.nanmax(relative_difference) * 100

In [ ]:
pep_processor = IsotonicPEP()
results_df = pd.DataFrame(index=[label_mapping[m]['label'] for m in methods])
for file_path, dataset_label in file_mapping.items():
    df_target = pd.read_csv(file_path, sep="\t")
    df_decoy = pd.read_csv(file_path.replace("target", "decoy"), sep="\t")
    df_target = df_target.drop(columns=columns_to_drop)
    df_decoy = df_decoy.drop(columns=columns_to_drop)
    df_target = df_target.rename(columns={"posterior_error_prob": "spline"})
    df_decoy = df_decoy.rename(columns={"posterior_error_prob": "spline"})

    df_target["d2pep_pava"], df_target["q_d2pep_pava"] = pep_processor.pep_regression(
        target=df_target["score"].values,
        decoy=df_decoy["score"].values,
        method="d2pep",
        regression_algo="PAVA"
    )

    df_target["d2pep_ispline"], df_target["q_d2pep_ispline"] = pep_processor.pep_regression(
        target=df_target["score"].values,
        decoy=df_decoy["score"].values,
        method="d2pep",
        regression_algo="ispline"
    )

    df_target["q2pep_pava"], df_target["q_q2pep_pava"] = pep_processor.pep_regression(
        q_values=df_target["q-value"].values,
        method="q2pep",
        regression_algo="PAVA"
    )

    df_target["q2pep_ip_ispline"], df_target["q_q2pep_ip_ispline"] = pep_processor.pep_regression(
        q_values=df_target["q-value"].values,
        method="q2pep",
        regression_algo="PAVA",
        ip=True,
        ip_algo="ispline"
    )

    df_target["q2pep_ip_pchip"], df_target["q_q2pep_ip_pchip"] = pep_processor.pep_regression(
        q_values=df_target["q-value"].values,
        method="q2pep",
        regression_algo="PAVA",
        ip=True,
        ip_algo="pchip"
    )

    df_target["q2pep_ispline"], df_target["q_q2pep_ispline"] = pep_processor.pep_regression(
        q_values=df_target["q-value"].values,
        method="q2pep",
        regression_algo="ispline"
    )

    df_target = estimate_q(df_target, "spline")
    q_values = df_target["q-value"].values
    file_results = {}
    for m in methods:
        max_diff = cal_max_rel_diff(q_values, df_target[f"q_{m}"].values)
        file_results[label_mapping[m]['label']] = max_diff
    results_df[dataset_label] = pd.Series(file_results)

    score = df_target["score"].values
    total_targets = len(df_target)
    targets_under_1perc_q = len(df_target[df_target["q-value"] < 0.01])
    a = np.linspace(0, 1, 100)
    b = a / 10 ** 0.25
    c = a * 10 ** 0.25
    plt.figure(figsize=(8, 8), dpi=600)
    for m in ["d2pep_pava", "d2pep_ispline"]:
        plt.plot(score, df_target[m], label=label_mapping[m]['label'], linewidth=3, linestyle=label_mapping[m]['linestyle'])
    plt.text(0.58, 0.9, f"All targets: {total_targets}", transform=plt.gca().transAxes, fontsize=18, fontweight='bold')
    plt.text(0.58, 0.8, f"q<0.01: {targets_under_1perc_q}", transform=plt.gca().transAxes, fontsize=18, fontweight='bold')
    plt.xlabel("Target scores", fontsize=24)
    plt.ylabel("Estimated PEPs", fontsize=24)
    plt.title(f"$D$ $prob.$-based method: {dataset_label}", fontsize=24)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.xlim(-1.2, 1)
    plt.legend(loc="lower left", fontsize=24)
    plt.savefig(f"figs/ispline/fd_pep_{dataset_label}.png",bbox_inches="tight")
    plt.show()

    plt.figure(figsize=(8, 8), dpi=600)
    cnt=0
    for m in ["d2pep_pava", "d2pep_ispline"]:
        max_diff = cal_max_rel_diff(q_values, df_target[f"q_{m}"].values)
        plt.text(0.05, 0.8 - cnt * 0.1, f"{label_mapping[m]['label']}: {max_diff:.0f}%", transform=plt.gca().transAxes, fontsize=18, fontweight='bold')
        cnt += 1
        plt.plot(df_target["q-value"], df_target[f"q_{m}"], label=label_mapping[m]['label'], linestyle=label_mapping[m]['linestyle'], linewidth=3)
    plt.text(0.05, 0.9, f"max relative difference:", transform=plt.gca().transAxes, fontsize=18, fontweight='bold')
    plt.plot(a, b, c="k", linewidth=1, linestyle="--")
    plt.plot(a, c, c="k", linewidth=1, linestyle="--")
    plt.plot(a, a, c="k", linewidth=1)
    plt.xlabel("FDR-derived $q$ values", fontsize=24)
    plt.ylabel("PEPs-derived $q$ values", fontsize=24)
    plt.title(f"$D$ $prob.$-based method: {dataset_label}", fontsize=24)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.legend(loc='lower right', fontsize=24)
    plt.xscale("log")
    plt.yscale("log")
    plt.xlim(0.00001, 1)
    plt.ylim(0.00001, 1)
    plt.savefig(f"figs/ispline/fd_q_{dataset_label}.png",bbox_inches="tight")
    plt.show()

    plt.figure(figsize=(8, 8), dpi=600)
    for m in ["q2pep_pava", "q2pep_ip_ispline", "q2pep_ispline"]:
        plt.plot(score, df_target[m], label=label_mapping[m]['label'], linewidth=3, linestyle=label_mapping[m]['linestyle'])
    plt.text(0.58, 0.9, f"All targets: {total_targets}", transform=plt.gca().transAxes, fontsize=18, fontweight='bold')
    plt.text(0.58, 0.8, f"q<0.01: {targets_under_1perc_q}", transform=plt.gca().transAxes, fontsize=18, fontweight='bold')
    plt.xlabel("Target scores", fontsize=24)
    plt.ylabel("Estimated PEPs", fontsize=24)
    plt.title(f"$q$ value-based method: {dataset_label}", fontsize=24)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.xlim(-1.2, 1)
    plt.legend(loc="lower left", fontsize=24)
    plt.savefig(f"figs/ispline/q_pep_{dataset_label}.png",bbox_inches="tight")
    plt.show()

    plt.figure(figsize=(8, 8), dpi=600)
    cnt=0
    for m in ["q2pep_pava", "q2pep_ip_ispline", "q2pep_ispline"]:
        max_diff = cal_max_rel_diff(q_values, df_target[f"q_{m}"].values)
        plt.text(0.05, 0.8 - cnt * 0.1, f"{label_mapping[m]['label']}: {max_diff:.0f}%", transform=plt.gca().transAxes, fontsize=18, fontweight='bold')
        cnt += 1
        plt.plot(df_target["q-value"], df_target[f"q_{m}"], label=label_mapping[m]['label'], linestyle=label_mapping[m]['linestyle'], linewidth=3)
    plt.text(0.05, 0.9, f"max relative difference:", transform=plt.gca().transAxes, fontsize=18, fontweight='bold')
    plt.plot(a, b, c="k", linewidth=1, linestyle="--")
    plt.plot(a, c, c="k", linewidth=1, linestyle="--")
    plt.plot(a, a, c="k", linewidth=1)
    plt.xlabel("FDR-derived $q$ values", fontsize=24)
    plt.ylabel("PEPs-derived $q$ values", fontsize=24)
    plt.title(f"$D$ $prob.$-based method: {dataset_label}", fontsize=24)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.legend(loc='lower right', fontsize=24)
    plt.xscale("log")
    plt.yscale("log")
    plt.xlim(0.00001, 1)
    plt.ylim(0.00001, 1)
    plt.savefig(f"figs/ispline/q_q_{dataset_label}.png",bbox_inches="tight")
    plt.show()
formatted_df = results_df.applymap(lambda x: f"{x:.0f}%")
formatted_df.to_csv("figs/ispline/max_diff.scipy.txt", sep="\t", index=True)